In [72]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [73]:
def get_mnist2():
    x, y = lib.datasets.get_mnist_train()
    train_y = y % 2
    train_x = torch.reshape(x, (len(x), -1))
    x, y = lib.datasets.get_mnist_test()
    test_y = y % 2
    test_x = torch.reshape(x, (len(x), -1))
    return train_x, train_y

dataset_func = lib.datasets.get_odd

if "train_x" not in globals():
    train_x, train_y = dataset_func()



In [78]:
def create_optimizer(net: nn.Module):
    if net.__class__.__name__ == "LiveNet":
        print("LiveNet")
        net: lib.livenet.LiveNet
        optimizer = lib.livenet.LiveNetOptimizer(net)
        # optimizer = torch.optim.Adam(net.parameters())
    else:
        print("Torch")
        optimizer = lib.optimizer.optimizer_with_lr_property(torch.optim.SGD, net.parameters(), lr=0.01)
        # optimizer = lib.optimizer.optimizer_with_lr_property(torch.optim.Adam, net.parameters(), betas=(0.0, 0.95))
    return optimizer


In [81]:
lib.utils.set_seed()
# network = lib.nets.ODD()
network = lib.nets.create_livenet_odd()
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=len(train_x))
criterion = lib.nets.criterion_1
optimizer = create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=30, adaptive_lr=False)


LiveNet


In [82]:

trainer.step(200)


Iˈ0.000 0.310 lib/trainer.py:89
Iˈ0.022 0.298 lib/trainer.py:89
Iˈ0.040 0.277 lib/trainer.py:89
Iˈ0.058 0.260 lib/trainer.py:89
Iˈ0.075 0.246 lib/trainer.py:89
Iˈ0.092 0.233 lib/trainer.py:89
Iˈ0.109 0.223 lib/trainer.py:89


In [51]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.3333

In [52]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_picker0(param):
    val0 = param["params"]["linear1.weight"][0][0].item()
    val1 = param["params"]["linear1.weight"][1][0].item()
    return val1 + val0

def param_picker1(param):
    val0 = param["params"]["linear1.weight"][0][1].item()
    return val0
    return param["grads"]["linear1.weight"][0][0].item() / 1000
    # return np.max(np.abs(param[0].numpy()))

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
# values = get_param_values(trainer.history, param_picker0)
# plt.plot(values)
# values = get_param_values(trainer.history, param_picker1)
# plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
accum.plot()




Using matplotlib backend: TkAgg


In [39]:
def form_balanced(x, y, n):
    assert n % 2 == 0
    n //= 2
    y = y.numpy()
    x = x.numpy()
    inds = np.argsort(y.squeeze(1), axis=0)
    y = y[inds]
    x = x[inds]
    i = 0
    batches_x = []
    batches_y = []
    while True:
        a = y[i:i + n]
        b = y[len(y) - i - n: len(y) - i]
        if not (a==0).all() or not (b==1).all():
            break
        batches_x += [x[i:i+n], x[len(y) - i - n: len(y) - i]]
        batches_y += [a, b]
        i += n

    x = np.vstack(batches_x)
    y = np.vstack(batches_y)
    return torch.tensor(x), torch.tensor(y)

train_x, train_y = form_balanced(train_x, train_y, 1000)
